In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['drug_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['drug_six_month'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [5]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.6362988082752609, 0.6329655135404522)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_six_month'].values

## columns 
cols = KY_X.columns

In [7]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 0.08, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge21.0',
 'age_at_current_charge23.0',
 'age_at_current_charge27.0',
 'age_at_current_charge36.0',
 'age_at_current_charge39.0',
 'age_at_current_charge49.0',
 'age_at_current_charge50.0',
 'age_at_current_charge52.0',
 'age_at_current_charge69.0',
 'age_at_current_charge70.0',
 'p_arrest4',
 'p_arrest6',
 'p_arrest8',
 'p_charges1',
 'p_charges2',
 'p_misdemeanor1',
 'p_misdemeanor2',
 'p_misdemeanor3',
 'p_property2',
 'p_property3',
 'p_weapon1',
 'p_felassult1',
 'p_misdeassult1',
 'p_drug2',
 'p_drug3',
 'p_dui1',
 'p_fta_two_year1',
 'p_pending_charge5',
 'p_probation2',
 'p_probation3',
 'six_month1',
 'one_year1']

In [8]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary['best_params'], stumps_summary['KY_score'], stumps_summary['auc_diffs']

({'C': 0.04}, 0.628925312199338, 0.08219604638520384)

### RiskSLIM

In [10]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_six_month'].values

## columns 
cols = KY_X.columns

In [16]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 
                                        0.05, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

21

In [17]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [18]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'drug_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000, 
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 04:10 PM | 263 rows in lookup table
11/12/19 @ 04:10 PM | ------------------------------------------------------------
11/12/19 @ 04:10 PM | runnning initialization procedure
11/12/19 @ 04:10 PM | ------------------------------------------------------------
11/12/19 @ 04:10 PM | CPA produced 2 cuts
11/12/19 @ 04:10 PM | running naive rounding on 6 solutions
11/12/19 @ 04:10 PM | best objective value: 0.5685
11/12/19 @ 04:10 PM | rounding produced 5 integer solutions
11/12/19 @ 04:10 PM | best objective value is 0.4304
11/12/19 @ 04:10 PM | running sequential rounding on 6 solutions
11/12/19 @ 04:10 PM | best objective value: 0.5685
11/12/19 @ 04:10 PM | sequential rounding produced 3 integer solutions
11/12/19 @ 04:10 PM | best objective value: 0.2998
11/12/19 @ 04:10 PM | polishing 8 solutions
11/12/19 @ 04:10 PM | best objective value: 0.2998
11/12/19 @ 04:10 PM | polishing produced 5 integer solutions
11/12/19 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 04:10 PM | adding 418 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1542.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 257284 71499        cutoff              0.1490        0.1418  1665835    4.80%           rho_8 N 257284 257283     35
 263307 72669        cutoff              0.1490        0.1419  1697625    4.73%        alpha_20 D 263307 263306     35
 269280 73804        0.1465    10        0.1490        0.1420  1728455    4.67%          rho_16 D 269280 269279     33
 275203 75037        cutoff              0.1490        0.1421  1759985    4.60%          rho_12 D 275203 275201     27
 280880 75941        cutoff              0.1490        0.1422  1791866    4.53%          rho_19 N 280880 280879     52
 286757 76819        0.1432    14        0.1490        0.1423  1823469    4.46%           rho_6 U 286757 213749     33
 292569 77666        0.1441    19        0.1490        0.1424  1854133    4.39%           rho_0 U 292569 108645     18
 298481 78730        0.1453    12        0.1490        0.1425  1884724    4.33%           rho_3 D 298481 298479     31
 304400 79685        0.1468    14        0.1490 

 1019045 65725        cutoff              0.1487        0.1473  4973157    0.93%           rho_3 D 1019045 1019043     40
 1025492 65195        cutoff              0.1487        0.1473  4994578    0.92%          rho_19 U 1025492 1025491     53
Elapsed time = 510.06 sec. (625949.46 ticks, tree = 24.14 MB, solutions = 13)
 1031625 64346        0.1474    13        0.1487        0.1473  5016178    0.90%           rho_9 U 1031625 1031624     48
 1038040 62743        cutoff              0.1487        0.1474  5034827    0.87%        alpha_20 D 1038040 1038039     46
 1044672 62591        cutoff              0.1487        0.1474  5052329    0.86%           rho_3 D 1044672 1044670     46
 1050783 60919        cutoff              0.1487        0.1474  5070199    0.83%           rho_5 D 1050783 540211     48
 1057104 59806        0.1477     8        0.1487        0.1475  5087412    0.82%           rho_3 U 1057104 1057103     44
 1063202 57866        0.1481    10        0.1487        0.1475  51052

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 04:20 PM | adding 395 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1514.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

...

 1827749 317825        cutoff              0.1469        0.1438  9259862    2.16%           rho_4 U 1827749 1693577     38
 1832970 317139        0.1458    14        0.1469        0.1438  9287850    2.15%           rho_3 D 1832970 1832969     32
 1838795 315996        0.1442    13        0.1469        0.1438  9310576    2.15%           rho_3 U 1838795 911783     34
 1844429 315474        cutoff              0.1469        0.1438  9335320    2.14%         alpha_1 D 1844429 1844428     51
 1850328 315201        0.1458    10        0.1469        0.1438  9360220    2.13%           rho_7 D 1850328 1850327     42
 1856198 315259        cutoff              0.1469        0.1438  9384351    2.12%         alpha_1 U 1856198 1856196     33
Elapsed time = 690.38 sec. (1121918.26 ticks, tree = 108.21 MB, solutions = 9)
 1862277 315543        0.1446    12        0.1469        0.1438  9409054    2.11%           rho_9 U 1862277 508963     37
 1867936 314646        0.1439    10        0.1469        0.143

 2553424 242578        cutoff              0.1469        0.1451 12065354    1.27%           rho_0 D 2553424 2553422     37
 2559077 240513        cutoff              0.1469        0.1451 12085177    1.26%          rho_11 D 2559077 1461665     23
Elapsed time = 936.91 sec. (1579735.02 ticks, tree = 87.62 MB, solutions = 9)
 2564981 238568        0.1456    12        0.1469        0.1451 12105399    1.25%           rho_5 N 2564981 2564980     39
 2571128 237389        0.1451    11        0.1469        0.1451 12126730    1.24%           rho_0 U 2571128 929665     26
 2577054 234929        cutoff              0.1469        0.1451 12145743    1.23%          rho_10 D 2577054 2349321     41
 2582674 233217        cutoff              0.1469        0.1451 12166554    1.22%        alpha_19 D 2582674 2582673     31
 2588723 231998        cutoff              0.1469        0.1452 12185933    1.21%          rho_15 U 2588723 988699     31
 2594219 229505        0.1452    11        0.1469        0.1452

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 05:04 PM | adding 372 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1585.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 344255 115848        0.1501    15        0.1523        0.1460  2136394    4.15%          rho_11 D 344255 162324     25
 352905 118467        0.1523     7        0.1523        0.1460  2180164    4.13%        alpha_20 N 352905 352903     39
 361044 120305        0.1486    10        0.1523        0.1461  2220404    4.09%           rho_4 D 361044 361042     42
 369135 122373        0.1511    11        0.1523        0.1461  2261966    4.05%          rho_12 N 369135 369134     37
 377576 124630        0.1479    15        0.1523        0.1462  2302719    4.01%           rho_6 U 377576 377574     31
 385880 126732        0.1480    15        0.1523        0.1463  2342634    3.98%        alpha_21 U 385880 349523     44
Elapsed time = 116.59 sec. (168131.69 ticks, tree = 37.26 MB, solutions = 8)
 394205 128729        cutoff              0.1523        0.1463  2382785    3.94%          rho_10 U 394205 171812     35
 402447 130640        0.1473    12        0.1523        0.1464  2423033    3.91%   

 1326609 198194        cutoff              0.1523        0.1499  6562693    1.61%        alpha_19 D 1326609 1326607     27
 1334101 198558        0.1514    14        0.1523        0.1499  6589398    1.60%           rho_0 U 1334101 1334100     38
 1341421 197530        cutoff              0.1523        0.1499  6616285    1.58%          rho_10 U 1341421 1341420     30
 1348890 196587        cutoff              0.1523        0.1499  6645096    1.56%         alpha_1 D 1348890 1348888     33
 1356034 195316        0.1517    15        0.1523        0.1499  6672797    1.55%           rho_6 N 1356034 1191408     31
 1363765 195213        0.1518    12        0.1523        0.1500  6699442    1.54%          rho_10 D 1363765 1363762     41
 1371228 194493        0.1500    14        0.1523        0.1500  6729837    1.52%          rho_15 U 1371228 167307     17
 1378710 194327        0.1514    16        0.1523        0.1500  6762952    1.51%          rho_11 D 1378710 1366332     29
 1386115 193284  

In [19]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_validation_score'])

(0.6202518064448579, 0.6700622082433336)

In [20]:
#### save results
summary_drug6_fl_inter_model = {"CART": cart_summary,
                                "EBM": ebm_summary,
                                "Stumps": stumps_summary,
                                "RiskSLIM": riskslim_summary}

In [21]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Six Month\\"
results = [["Drug",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Six Month FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)